In [ ]:
import pylab
import pandas as pd
import numpy as np
import os
from scipy import stats
from scipy.stats import ks_2samp
from sklearn.mixture import GaussianMixture
from scipy.optimize import minimize

def dir_path(sPath):
    total_path = []

    def print_dir_contents(sPath):
        nonlocal total_path

        for schild in os.listdir(sPath):
            sChildPath = os.path.join(sPath, schild)
            if os.path.isdir(sChildPath):
                print_dir_contents(sChildPath)
            elif schild.endswith('.csv'):
                total_path.append(sChildPath)

        return total_path

    total_path = print_dir_contents(sPath)
    return total_path

def em_dweibull(data, max_iter=100, tol=1e-4):
    def log_likelihood(params, data):
        c, loc, scale = params
        return -np.sum(stats.dweibull.logpdf(data, c, loc, scale))
    c_init = 1.0
    loc_init = np.mean(data)
    scale_init = np.std(data)
    params_init = np.array([c_init, loc_init, scale_init])
    
    result = minimize(log_likelihood, params_init, args=(data,), method='L-BFGS-B',
                      bounds=[(0.1, None), (None, None), (0.1, None)])

    c, loc, scale = result.x
    return c, loc, scale

def gaussian_mixture_distribution(csv_path):
    mydata = pd.read_csv(csv_path)
    mydata = mydata.T
    mydata.columns = mydata.iloc[0]
    mydata = mydata.drop(mydata.index[0])
    mydata = mydata.astype('float64')

    all_dists_df = pd.DataFrame(columns=['gene', 'Distribution', 'param', 'sumsquare_error', 'aic', 'bic', 'ks', 'ks_pvalue'])
    for gene in list(mydata.columns):
        data = mydata[gene]
        if (data==0).all() == False:
            data_2d = data.values.reshape(-1, 1)

            dis_num = np.array([5])  
            results = []
            for n in dis_num:
                gmm = GaussianMixture(n_components=n, covariance_type='full', max_iter=100, tol=1e-4)
                gmm.fit(data_2d)
                param = gmm.get_params()
                sse = pylab.sum((gmm.predict_proba(data_2d) - data_2d) ** 2)
                aic = gmm.aic(data_2d)
                bic = gmm.bic(data_2d)
                ks_statistic, p_value = ks_2samp(data_2d[:, 0], gmm.sample(len(data_2d))[0][:, 0])
                results.append((gene, n, param, sse, aic, bic, ks_statistic, p_value))

            df_results = pd.DataFrame(results, columns=['gene', 'Distribution', 'param', 'sumsquare_error', 'aic', 'bic', 'ks', 'ks_pvalue']).sort_values('bic')
            all_dists_df = pd.concat([all_dists_df, df_results], axis=0)

    all_dists_df = all_dists_df.reset_index(drop=True)
    return all_dists_df

def set_distribution(csv_path):
    mydata = pd.read_csv(csv_path)
    mydata = mydata.T
    mydata.columns = mydata.iloc[0]
    mydata = mydata.drop(mydata.index[0])
    mydata = mydata.astype('float64')

    all_dists_df = pd.DataFrame(columns=['gene', 'Distribution', 'param', 'sumsquare_error', 'aic', 'bic', 'ks', 'ks_pvalue'])
    for gene in list(mydata.columns):
        data = mydata[gene]
        if (data==0).all() == False:
            results = []
            dist_names = ['norm', 't', 'pareto', 'genextreme', 'laplace', 'cauchy', 'chi2', 'expon', 'exponpow', 'gamma', 'beta', 'lognorm', 'loggamma', 'uniform']
            for dist_name in dist_names:
                dist = getattr(stats, dist_name)
                param = dist.fit(data)
                log_likelihood = np.sum(dist.logpdf(data, *param))
                pdf_fitted = dist.pdf(data, *param[:-2], loc=param[-2], scale=param[-1])
                sse = np.sum(np.power(pdf_fitted - data, 2.0))
                n = len(data)
                k = len(param)
                aic = -2 * log_likelihood + 2 * k
                bic = -2 * log_likelihood + k * np.log(len(data))
                ks_test, p_value = stats.kstest(data, dist_name, args=param)
                results.append((gene, dist_name, param, sse, aic, bic, ks_test, p_value))

            # EM for dweibull
            dweibull_param = em_dweibull(data)
            log_likelihood = np.sum(stats.dweibull.logpdf(data, *dweibull_param))
            pdf_fitted = stats.dweibull.pdf(data, *dweibull_param)
            sse = np.sum(np.power(pdf_fitted - data, 2.0))
            n = len(data)
            k = len(dweibull_param)
            aic = -2 * log_likelihood + 2 * k
            bic = -2 * log_likelihood + k * np.log(len(data))
            ks_test, p_value = stats.kstest(data, 'dweibull', args=dweibull_param)
            results.append((gene, 'dweibull_em', dweibull_param, sse, aic, bic, ks_test, p_value))

            df_results = pd.DataFrame(results, columns=['gene', 'Distribution', 'param', 'sumsquare_error', 'aic', 'bic', 'ks', 'ks_pvalue']).sort_values('bic')
            all_dists_df = pd.concat([all_dists_df, df_results], axis=0)

    all_dists_df = all_dists_df.reset_index(drop=True)
    return all_dists_df

def identify_sample(num_greater_than_2nd_col, num_greater_than_1st_col):
    if num_greater_than_2nd_col > num_greater_than_1st_col:
        return 'cancer'
    else:
        return 'normal'
    

def laplace_smoothing(x):
    return (x + 1) / (x.sum() + len(x))    
    
import pandas as pd
from sklearn.model_selection import train_test_split

def split_data(data_path):
    data = pd.read_csv(data_path, index_col=0)
    cancer_samples = data.columns[:126]
    normal_samples = data.columns[126:]
    test_size_temp= 0.2
    labels = pd.DataFrame({'label': [1] * len(cancer_samples) + [0] * len(normal_samples)},
                          index=cancer_samples.tolist() + normal_samples.tolist())
    cancer_train, cancer_test = train_test_split(cancer_samples, test_size=test_size_temp)
    normal_train, normal_test = train_test_split(normal_samples, test_size=test_size_temp)
    train_samples = list(cancer_train) + list(normal_train)
    test_samples = list(cancer_test) + list(normal_test)
    train_data = data.loc[:, train_samples]
    test_data = data.loc[:, test_samples]
    test_labels = labels.loc[test_samples]
    cancer_train_data = data.loc[:, cancer_train]
    normal_train_data = data.loc[:, normal_train]
    cancer_test_data = data.loc[:, cancer_test]
    normal_test_data = data.loc[:, normal_test]
    return train_data, test_data, test_labels, cancer_train_data, normal_train_data, cancer_test_data,normal_test_data

f1score_and_youdenindex = []

for i in range(50):
    train_data, test_data, test_labels,cancer_train_data, normal_train_data, cancer_test_data,normal_test_data = split_data(r"D:\WORKSPACE2\...")
    #train_data.to_csv('train_data.csv')
    test_data_name = str(i)+ '_test_data.csv'
    test_data.to_csv(test_data_name)
    
    test_labels_name = str(i)+ '_labels_data.csv'
    test_labels.to_csv(test_labels_name)
    
    cancer_train_data_name = str(i)+ '_cancer_train_data.csv'
    cancer_train_data.to_csv(cancer_train_data_name)
    
    normal_train_data_name = str(i)+ 'normal_train_data.csv'
    normal_train_data.to_csv( normal_train_data_name)
    #cancer_test_data.to_csv('cancer_test_data.csv')
    #normal_test_data.to_csv('normal_test_data.csv')



    wait_for_clust = test_data
    wait_for_clust = wait_for_clust.T
    wait_for_clust = wait_for_clust.astype('float64')
    wait_for_clust


    wait_clust_cancer_data = wait_for_clust

    csv_path_cancer = cancer_train_data
    gaussian_best_dist_df_cancer = gaussian_mixture_distribution(csv_path_cancer,wait_clust_cancer_data,0.5)    
    other_best_dist_df_cancer = set_distribution(csv_path_cancer,wait_clust_cancer_data,0.5)
    #print(other_best_dist_df)
    all_dist_cancer = pd.concat([gaussian_best_dist_df_cancer,other_best_dist_df_cancer],axis=0)
    #print(all_dist_cancer)
    all_dist_cancer.reset_index(inplace=True, drop=True)
    finally_data_cancer = pd.DataFrame(index=['gene', 'Distribution', 'sumsquare_error', 'aic', 'bic', 'ks', 'ks_pvalue','lastresult'])
    grouped = all_dist_cancer.groupby('gene')
    for name, group in grouped:
        result = group.sort_values('bic').iloc[0]
        finally_data_cancer = pd.concat([finally_data_cancer,result],axis=1)

    finally_data_cancer = finally_data_cancer.T
    out_path_cancer = str(i)+'_train_dist_result_cancer.xlsx'
    finally_data_cancer.to_excel(out_path_cancer)

        
                
    wait_clust_normal_data = wait_for_clust
    csv_path_normal = normal_train_data
    gaussian_best_dist_df_normal = gaussian_mixture_distribution(csv_path_normal,wait_clust_normal_data,0.5)
    other_best_dist_df_normal = set_distribution(csv_path_normal,wait_clust_normal_data,0.5)
    #print(other_best_dist_df)
    all_dist_normal = pd.concat([gaussian_best_dist_df_normal,other_best_dist_df_normal],axis=0)
    #print(all_dist_normal)
    all_dist_normal.reset_index(inplace=True, drop=True)

    finally_data_normal = pd.DataFrame(index=['gene', 'Distribution', 'sumsquare_error', 'aic', 'bic', 'ks', 'ks_pvalue','lastresult'])
    grouped = all_dist_normal.groupby('gene')
    for name, group in grouped:
        result = group.sort_values('bic').iloc[0]
        finally_data_normal = pd.concat([finally_data_normal,result],axis=1)
    finally_data_normal = finally_data_normal.T
    out_path_normal = str(i)+'_train_dist_result_normal.xlsx'
    finally_data_normal.to_excel(out_path_normal)



    finally_data_cancer_filtered = finally_data_cancer[finally_data_cancer['ks_pvalue'] > 0.01]
    finally_data_normal_filtered = finally_data_normal[finally_data_normal['ks_pvalue'] > 0.01]
    cancer_genes = set(finally_data_cancer_filtered.iloc[:, 0].tolist())
    normal_genes = set(finally_data_normal_filtered.iloc[:, 0].tolist())


    common_genes = list(cancer_genes & normal_genes)


    filtered_cancer_data = finally_data_cancer_filtered[finally_data_cancer_filtered.iloc[:, 0].isin(common_genes)]
    filtered_normal_data = finally_data_normal_filtered[finally_data_normal_filtered.iloc[:, 0].isin(common_genes)]

    filtered_cancer_data = filtered_cancer_data.set_index(filtered_cancer_data.columns[0])
    filtered_normal_data = filtered_normal_data.set_index(filtered_normal_data.columns[0])

    posterior_probability_cancer =  filtered_cancer_data['lastresult']
    posterior_probability_normal = filtered_normal_data['lastresult']
    posterior_probability_cancer = posterior_probability_cancer.apply(lambda x: ','.join(map(str, x))).str.split(',', expand=True)
    posterior_probability_normal = posterior_probability_normal.apply(lambda x: ','.join(map(str, x))).str.split(',', expand=True)

    row_names = wait_for_clust.index.tolist()


    posterior_probability_cancer.columns = row_names

    out_path_cancer_posterior = str(i)+'_posterior_probability_cancer.xlsx'
    posterior_probability_cancer.to_excel(out_path_cancer_posterior)

    posterior_probability_normal.columns = row_names

    out_path_normal_posterior = str(i)+'_posterior_probability_normal.xlsx'
    posterior_probability_normal.to_excel(out_path_normal_posterior)

    column_names = posterior_probability_cancer.columns


    sample_type=[]
    for col_name in column_names:

        
        col_cancer = posterior_probability_cancer[col_name].astype('float64')
        col_cancer = np.add(col_cancer, 0.000000001)#
        multiplied_col_cancer = np.multiply.reduce(col_cancer)*0.5
        col_normal = posterior_probability_normal[col_name].astype('float64')
        col_normal = np.add(col_normal, 0.000000001)#
        multiplied_col_normal = np.multiply.reduce(col_normal)*0.5
        greater_than = [multiplied_col_cancer >= multiplied_col_normal]
        less_than = [multiplied_col_cancer < multiplied_col_normal]
        # 统计两列数字中每列数字比另一列数字大的数量
        num_greater_than_2nd_col = sum(greater_than)
        num_greater_than_1st_col = sum(less_than)   
        wait_clust_data_type = identify_sample(num_greater_than_2nd_col, num_greater_than_1st_col)
        sample_type.append((col_name,wait_clust_data_type,num_greater_than_2nd_col,num_greater_than_1st_col))
    sample_type_df=pd.DataFrame(sample_type, columns=['sample_name','sample_type','癌症样本基因后验概率比正常样本基因后验概率大的数量','癌症样本基因后验概率比正常样本基因后验概率小的数量'])
    sample_type_df_name = str(i)+'_test_sample_type_df.xlsx' 
    sample_type_df.to_excel(sample_type_df_name)


    from sklearn.metrics import accuracy_score
    from sklearn.metrics import f1_score
    from sklearn.metrics import confusion_matrix
    sample_type_df['sample_type'] = sample_type_df['sample_type'].replace({'cancer': 1, 'normal': 0})
    sample_type_df_label = sample_type_df.loc[:,'sample_type']
    sample_type_df_label=sample_type_df_label.values
    sample_type_df_label
    ###Y_TEST
    true_sample_type_df_label = test_labels
    #true_sample_type_df_label=true_sample_type_df_label.T
    true_sample_type_df_label=true_sample_type_df_label.values
 
    f1 = f1_score(true_sample_type_df_label, sample_type_df_label, average='weighted')
    print("F1 score:", f1)



    cm = confusion_matrix(true_sample_type_df_label, sample_type_df_label)
    tn, fp, fn, tp = cm.ravel()
    youden = tp/(tp+fn) - fp/(fp+tn)
    print("Youden index:", youden)
    #f1score_and_youdenindex[i]=[f1,youden]
    f1score_and_youdenindex .append((f1,youden))
f1score_and_youdenindex = pd.DataFrame(f1score_and_youdenindex,columns=['F1_score', 'Youden_Index'])
f1score_and_youdenindex_name =  'f1score_and_youdenindex.xlsx'
f1score_and_youdenindex.to_excel(f1score_and_youdenindex_name)